In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import re

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
  
        title = soup.find("h1", attrs={"class":'Description___StyledH-sc-82a36a-2 bofYPK'})
        title_value = title.text
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

In [3]:
# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("td", attrs={'class':'line-through p-0'}).string.strip()

    except AttributeError:

        try:
            price = soup.find("td", attrs={'class':'Description___StyledTd-sc-82a36a-4 fLZywG'}).string.strip()

        except:
            price = ""

    return price

In [4]:
# Function to extract Product mrp
def get_mrp(soup):

    try:
        mrp = soup.find("td", attrs={"class":'line-through p-0'})
        mrp_value = mrp.text
        mrp_string = mrp_value.strip()

    except AttributeError:
        mrp_string = ""

    return mrp_string

In [5]:
# Function to extract Product Brand
def get_brand(soup):

    try:

        brand = soup.find("a", attrs={"class":'Description___StyledLink-sc-82a36a-1 gePjxR'})
        brand_value = brand.text
        brand_string = brand_value.strip()

    except AttributeError:
        brand_string = ""

    return brand_string

In [6]:
# Function to extract Product Quantity
def get_quantity(soup):

    try:

        quantity = soup.find("span", attrs={"class":'Label-sc-15v1nk5-0 PackSizeSelector___StyledLabel2-sc-l9rhbt-2 gJxZPQ hDJUsF'})
        quantity_value = quantity.text
        quantity_string = quantity_value.strip()

    except AttributeError:
        quantity_string = ""

    return quantity_string

In [7]:
# Function to extract Product Code
def get_product_code(url):
    pattern = re.compile(r'/pd/(\d+)/')
    match = pattern.search(url)
    if match:
        return match.group(1)
    else:
        return None

In [8]:
if __name__ == '__main__':


    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # List of URLs
    urls = [
        "https://www.bigbasket.com/cl/bakery-cakes-dairy/?nc=nb&page=8",
        "https://www.bigbasket.com/cl/foodgrains-oil-masala/?nc=nb",
        "https://www.bigbasket.com/cl/beverages/?nc=nb",
        "https://www.bigbasket.com/cl/snacks-branded-foods/?nc=nb",
        "https://www.bigbasket.com/cl/beauty-hygiene/?nc=nb"
    ]
    
    d = {"title":[], "price":[], "brand":[], "quantity":[], "product_code":[]}
    
    for url in urls:
        webpage = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(webpage.content, "html.parser")
        
        links = soup.find_all("a", class_='h-full')
        links_list = ["https://www.bigbasket.com" + link.get('href') for link in links]
        
        for link in links_list:
            new_webpage = requests.get(link, headers=HEADERS)
            new_soup = BeautifulSoup(new_webpage.content, "html.parser")
            
            d['title'].append(get_title(new_soup))
            d['price'].append(get_price(new_soup))
            d['brand'].append(get_brand(new_soup))
            d['quantity'].append(get_quantity(new_soup))
            d['product_code'].append(get_product_code(link))
            
            time.sleep(1)
    
    bb_df = pd.DataFrame.from_dict(d)
    bb_df['title'].replace('', np.nan, inplace=True)
    bb_df = bb_df.dropna(subset=['title'])
    bb_df.to_csv("bb.csv", header=True, index=False)

In [9]:
bb_df

,title,price,brand,quantity,product_code
0,"One Good Peanut Curd, 2 x 500 g Multipack",₹198,One Good,2 x 500 g,1229863
1,"Yakult Probiotic Fermented Milk Drink - Light,...",,Yakult,4x65 ml,1209280
2,"Nestle A+ Slim Dahi - Low-Fat Curd, 380 g Cup",₹85,Nestle A+,,206701
3,"Tasties Biscuits - Osmania, 2x200 g Multipack",₹238,Tasties,2x200 g,1203304
4,"Milky Mist Mozzarella - Shredded, Pizza, Chees...",₹160,Milky Mist,200 g,40206820
...,...,...,...,...,...
250,Gillette Venus Simply Venus 3 Blade Hair Remov...,₹80,Gillette Venus,1 pc,40154107
251,Vaseline Intensive Care Deep Moisture Body Lot...,₹1250,Vaseline,600 ml,1222120
252,Sensodyne Sensitive Toothbrush - With Soft Rou...,₹155,Sensodyne,4 pcs,40161042
253,STAYFREE Secure Cottony XL - Sanitary Pads For...,₹550,STAYFREE,2x40 pcs,1215820
